In [1]:
from datetime import datetime, timedelta
import requests
from algosdk import future, encoding, logic
from contracts.crowdfounding import utils
from algosdk.v2client import algod
from contracts.external_utils.auction_demo.testing import resources

import pprint

from algosdk.constants import MIN_TXN_FEE



In [2]:
# Creating a fake account to interact with the APIs
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
algod_client = algod.AlgodClient(algod_token, algod_address)
test_account = resources.getTemporaryAccount(algod_client)

print("Test account "+ test_account.getAddress() +  " with: " + str(algod_client.account_info(test_account.getAddress())["amount"]) + " microAlgos")

Test account JGXHWFGOZJ33EORBGC2S6XEDKYI4SQSPAXYPKD4MLKXAIZKENZPZ6MAIAQ with: 100000000 microAlgos


### Account information

In [3]:
query = {'address': test_account.getAddress()}
response = requests.get('http://localhost:8501/address_info', params=query)
pprint.pprint(response.json())


{'address': 'JGXHWFGOZJ33EORBGC2S6XEDKYI4SQSPAXYPKD4MLKXAIZKENZPZ6MAIAQ',
 'amount': 100000000,
 'amount-without-pending-rewards': 100000000,
 'apps-local-state': [],
 'apps-total-schema': {'num-byte-slice': 0, 'num-uint': 0},
 'assets': [],
 'created-apps': [],
 'created-assets': [],
 'min-balance': 100000,
 'pending-rewards': 0,
 'reward-base': 15,
 'rewards': 0,
 'round': 614,
 'status': 'Offline',
 'total-apps-opted-in': 0,
 'total-assets-opted-in': 0,
 'total-created-apps': 0,
 'total-created-assets': 0}


### Creating the contract

API EXAMPLE

 > http://localhost:8501/create_contract?sender=SUBSTITUTE_WITH_PUBLIC_KEY_&pool_name=prova&target=10000&startTime=1653775200&endTime=1653948000


In [4]:
startTime = int(datetime.timestamp(datetime.now() + timedelta(hours=1))) # pool starting from one hour from now
endTime = int(datetime.timestamp(datetime.now() + timedelta(hours=3))) # lasting two hours
pool_name = "usage_example_pool"
target = 12300000 # microAlgo

# We can notice that backend prepares transaction only and expects user to sign it

query = {"sender": test_account.getAddress(), "pool_name":pool_name,"target":target, "startTime": startTime, "endTime":endTime}
txt_to_sign_encoded = requests.get('http://localhost:8501/create_contract', params=query).json()

In [5]:
# Sign transaction

txt_to_sign = encoding.future_msgpack_decode(txt_to_sign_encoded)

signedTxn = txt_to_sign.sign(test_account.getPrivateKey())

query = {"contract_txn": encoding.msgpack_encode(signedTxn)}
app_id = requests.get('http://localhost:8501/sign_contract', params=query).json()
print(app_id)

{'application_id': 125}


In a webapp scenario the user can interact with the smart contract using either another API endpoint preparing the group transaction or signing it as the code in the cell below.

### Setup the contract

In [6]:
# Setup group transaction

params = algod_client.suggested_params()
params.fee = 2 * MIN_TXN_FEE
params.flat_fee = True

fundAppTxn = future.transaction.PaymentTxn(
            sender = test_account.getAddress(),
            receiver = logic.get_application_address(app_id["application_id"]),
            amt = 200000, 
            sp = algod_client.suggested_params(),
        )

setupTxn = future.transaction.ApplicationNoOpTxn(
            sender = test_account.getAddress(),
            index=app_id["application_id"],
            app_args=[b"setup"],
            sp= params
    )

future.transaction.assign_group_id([fundAppTxn, setupTxn])

# Sign the transaction

signedFundAppTxn = fundAppTxn.sign(test_account.getPrivateKey())
signedSetupTxn = setupTxn.sign(test_account.getPrivateKey())

# Send the transaction

setup_group_txn = algod_client.send_transactions([signedFundAppTxn, signedSetupTxn])
utils.inspect_transaction(setup_group_txn,algod_client)

Transaction information: {
    "confirmed-round": 620,
    "pool-error": "",
    "txn": {
        "sig": "vET0ULfLgzt+XKCDjj361uhbEkvSMF9a9SSQxE/TlaoE1AyB9L8UJNxubidK3o1uQXyYZvYhh8NJuEqZ+0+cDQ==",
        "txn": {
            "amt": 200000,
            "fee": 1000,
            "fv": 618,
            "gen": "sandnet-v1",
            "gh": "AKL3TWSUe2YYSAm7A0cZJUeeVQnw6c1FhKGXNrCMPTc=",
            "grp": "SVsmFWAiK0Dj1Owr9luLexdQ2l2E8IS1cCF5s8nY0eQ=",
            "lv": 1618,
            "rcv": "JOUMWHYDHUY54RA5GHVLZAQCTRDSJUNZJ4EPRNNYIWOFDTWQSZB4OHNR2Y",
            "snd": "JGXHWFGOZJ33EORBGC2S6XEDKYI4SQSPAXYPKD4MLKXAIZKENZPZ6MAIAQ",
            "type": "pay"
        }
    }
}
'note'


### Retrieving contracts information

In [7]:
query = {'address': test_account.getAddress()}
response = requests.get('http://localhost:8501/get_all_contracts', params=query)

all_crowdfund_pools = response.json()
active_pools = [pool for pool in all_crowdfund_pools if pool["endtime"]>datetime.timestamp(datetime.now())]
active_pools_except_test_account =  [pool for pool in active_pools if pool["app_id"]!=app_id["application_id"]]

end_pool_app_id = [pool["app_id"] for pool in active_pools_except_test_account if pool["endtime"]>datetime.timestamp(datetime.now())]

# Here we can see that the only active pools are one of the setup action and the one we built now
pprint.pprint(active_pools_except_test_account)

[{'app_id': 79,
  'create_tx_id': 'NRLXVBB54K7CV5NSHSWJKOSL6BHMQVKOTU2YXYZSBICPRRB5R3OQ            ',
  'endtime': 1655209591,
  'founder': 'TLKP3ILDGCAPB4WB5A4CMF5HWO74HK6N4YT4VA37QTAJGZ5R5WC6ODGX3U      ',
  'pool_name': 'pool_founder7444606                                             ',
  'starttime': 1655123196,
  'target': 10000000}]


Let's see other pool status

In [8]:
pool_data = algod_client.account_info(logic.get_application_address(end_pool_app_id[0]))
token_id = pool_data['assets'][0]['asset-id']

current_amount, total_amount = pool_data['assets'][0]["amount"], \
                 [created_asset["params"]["total"] for created_asset in pool_data[ 'created-assets'] if created_asset["index"]==token_id][0]
                 
print("Pool " + active_pools_except_test_account[0]["pool_name"].strip() +" percentage completion", 100*(1- current_amount/total_amount), "%")  

Pool pool_founder7444606 percentage completion 80.0 %


Other contract interactions can be found in the _scenario setup_